In [1]:
import sys
import os
# import re
# import gzip
import argparse
# import subprocess
import pandas as pd
import numpy as np

import utils

In [2]:
input_dir = '/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/'
save_dir = '/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/test_TreeAlign/'
datatag = 'SA535X4XB05462'
# expr_fn = os.path.join(input_dir,datatag+'_expr.csv.gz')
expr_fn = os.path.join(input_dir,datatag+'_expr_introns.csv.gz')
cnv_fn = os.path.join(input_dir,datatag+'_clones_cnv.csv.gz')
cell_clones_fn = os.path.join(input_dir,datatag+'_cell_clones.csv.gz')
tree_fn = None

print(expr_fn)
print(cnv_fn)
print(cell_clones_fn)


/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/SA535X4XB05462_expr_introns.csv.gz
/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/SA535X4XB05462_clones_cnv.csv.gz
/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/SA535X4XB05462_cell_clones.csv.gz


In [9]:
import logging
TreeAlign_mode = 'ddd'
from logging.handlers import TimedRotatingFileHandler
log_fn = os.path.join(save_dir, datatag+'_log_params.txt')
h = TimedRotatingFileHandler(log_fn)

logging.basicConfig(filename=os.path.join(save_dir, datatag+'_log_params.txt'), encoding='utf-8', level=logging.DEBUG, filemode='w')
logging.info(expr_fn)
logging.info(cnv_fn)
logging.info(datatag)
logging.info('TreeAlign_mode: ' + TreeAlign_mode)

# logging.info('min_clone_assign_prob: '+min_clone_assign_prob)
# logging.info(iterations+' iterations')
logging.info(datatag)
logging.info(save_dir)
# logging.info(f"The time of execution of replication timing program is : {td:.03f} mins")

In [7]:
# --expr_fn /home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/example_expr.csv --cnv_fn /home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/example_gene_cnv.csv --cell_clones_fn /home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/example_cell_clone.csv --datatag test --save_dir /home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/test_pipeline/

# input_dir = '/home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/'
# save_dir = '/home/htran/Projects/farhia_project/rnaseq/method_testing/TreeAlign/data/test_pipeline/'
# expr_fn = os.path.join(input_dir,'example_expr.csv')
# cnv_fn = os.path.join(input_dir,'example_gene_cnv.csv')
# cell_clones_fn = os.path.join(input_dir,'example_cell_clone.csv')
# tree_fn = None




'/home/htran/storage/rnaseq_datasets/testing_space/clonealign2/SA535X4XB05462/test_TreeAlign/SA535X4XB05462_log_params.txt'

In [4]:



input_dic = utils.load_input_data(expr_fn, cnv_fn, cell_clones_fn, tree_fn)

Loading expression data: 
(938, 1872)
                 AAACCCACACAGAGAC-1  AAACGCTTCCCGGTAG-1  AAAGAACCATCTGGGC-1  \
ENSG00000000457                   3                   0                   0   
ENSG00000001626                  17                   8                   5   

                 AAAGGATTCTTAATCC-1  AAAGGATTCTTCACAT-1  AAAGGGCGTTTGACAC-1  \
ENSG00000000457                   0                   0                   0   
ENSG00000001626                   0                   0                   0   

                 AAAGTCCTCATCTATC-1  AAAGTGAAGCTGTTAC-1  AAATGGAGTAACAGGC-1  \
ENSG00000000457                   1                   1                   0   
ENSG00000001626                   3                   3                   0   

                 AACAAAGCAACTCCAA-1  ...  TTCTAACGTGACGCCT-1  \
ENSG00000000457                   1  ...                   0   
ENSG00000001626                   1  ...                   0   

                 TTGATGGAGTATTGCC-1  TTGATGGGTTGCACGC-1

In [5]:
TreeAlign_mode = 'clonal_infer'
min_clone_assign_prob = 0.6
iterations = 500
datatag = 'test'
if cell_clones_fn is not None and TreeAlign_mode=='clonal_infer': 
    print('Run TreeAlign using clonal inference mode')
    utils.run_clone_TreeAlign(input_dic["expr"], input_dic["cnv"], input_dic["clone"], save_dir, datatag,
                    min_clone_assign_prob=min_clone_assign_prob, max_iter=iterations, min_clone_cell_count=1, cnv_cutoff=8)
else:
    print('Run TreeAlign using tree phylogenetic as input for inference')
    utils.run_phylo_TreeAlign(expr, cnv, tree, save_dir, datatag)

Run TreeAlign using clonal inference mode
Holala
gene count: 938
cell count: 1872
seed = 1
Start Inference.
ELBO converged at iteration 137
Not very czary


In [6]:

assigned_fn = os.path.join(save_dir, 'test_clone_assign.csv.gz')
assigned = pd.read_csv(assigned_fn, compression='gzip') #, index_col=0

assigned.head(3)

,cell_id,clone_id
0,AAACCCACACAGAGAC-1,I
1,AAACGCTTCCCGGTAG-1,A
2,AAAGAACCATCTGGGC-1,I


In [7]:
# str(assigned['clone_id'])
cls = [str(c) for c in assigned['clone_id'].values]
unique, counts = np.unique(cls, return_counts=True)
print(dict(zip(unique, counts)))

{'A': 500, 'C': 238, 'H': 629, 'I': 338, 'N': 156, 'nan': 11}
